In [1]:
import pandas as pd
import numpy as np
from os.path import join as joinpath
path = r"C:\Users\sdhec\Google Drive\Fantasy Football"

In [2]:
PlayerData = pd.read_excel(r"PlayerData_2019.xlsx")
Roster = pd.read_excel(r'RosterData_2020.xlsx')
TeamData = pd.read_excel('TeamData_2019.xlsx')

In [3]:
# Use the player's team from the 2020 Depth Chart Roster
# Compute the opportunity share for each position: How many rushing attempts or targets are there, and split them across players. 
    # When stating out, do not change the targets, rather the share. This way you explicitly have to change the Team Total to increase 
    # a player's projection. If it doesn't make sense, it puts a ceiling on what you can change. 
    
TeamVolume = (PlayerData
 [['TeamName','Player', 'Pos', 'Att', 'Rushing_Att', 'Rushing_Yds', 'Receiving_Tgt']]
 [PlayerData.Pos.isin(['QB', 'RB', 'WR', 'TE'])]
 .groupby(['TeamName', 'Pos'])
 .sum()
 .reset_index()
 .sort_values(['TeamName', 'Att', 'Rushing_Att', 'Receiving_Tgt'], ascending = [1,0,0, 0])
)

TeamVolume.G = 16
TeamVolume['PassAtt_G'] = TeamVolume.Att/TeamVolume.G
TeamVolume['RushAtt_G'] = TeamVolume.Rushing_Att/TeamVolume.G
TeamVolume['Targets_G'] = TeamVolume.Receiving_Tgt/TeamVolume.G
TeamVolume['PlaysPerGame'] = (TeamVolume.PassAtt_G + TeamVolume.RushAtt_G + TeamVolume.Targets_G)
TeamVolume['PassRunSplit'] = TeamVolume.PassAtt_G / TeamVolume.PlaysPerGame
    
    
PassingVolume = (
    PlayerData 
    [['TeamName','Player', 'Pos', 'G', 'Att', 'Cmp', 'Yds', 'TD', 'Int']]
    .groupby(['TeamName', 'Player', 'Pos'])
    .sum()
    .reset_index()

)
PassingVolume['PassAttTotal'] = (PassingVolume
                                 .merge(
                                     PassingVolume
                                     .groupby('TeamName')
                                     .Att
                                     .sum()
                                     .reset_index()
                                     .rename(columns = {'Att':'PassAttTotal'}), on = 'TeamName')
                                 ['PassAttTotal'])
PassingVolume['PassShare'] = (PassingVolume.Att )/ PassingVolume.PassAttTotal
PassingVolume['PassAtt_G'] = PassingVolume.Att/PassingVolume.G
PassingVolume['CmpPct'] = PassingVolume.Cmp / PassingVolume.Att
PassingVolume['Yds_Cmp'] = PassingVolume.Yds / PassingVolume.Cmp
PassingVolume['TD_Cmp'] = PassingVolume.TD / PassingVolume.Cmp
PassingVolume['Int_Att'] = PassingVolume.Int / PassingVolume.Att
PassingVolume = PassingVolume.sort_values(['PassAtt_G', 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att'], ascending = [0,0,0,0,1])


RushingVolume = (
    PlayerData
    [['TeamName','Player', 'Pos', 'G', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD']]
    .groupby(['TeamName', 'Player','Pos'])
    .sum()
    .reset_index()
)
RushingVolume['RushTotal'] = (RushingVolume
                                 .merge(
                                     RushingVolume
                                     .groupby('TeamName')
                                     .Rushing_Att
                                     .sum()
                                     .reset_index()
                                     .rename(columns = {'Rushing_Att':'RushTotal'}), on = 'TeamName')
                                 ['RushTotal'])
RushingVolume['RushShare'] = (RushingVolume.Rushing_Att )/ RushingVolume.RushTotal
RushingVolume['RushAtt_G'] = RushingVolume.Rushing_Att/RushingVolume.G
RushingVolume['Yds_Rush'] = RushingVolume.Rushing_Yds/RushingVolume.Rushing_Att
RushingVolume['TD_Rush'] = RushingVolume.Rushing_TD/RushingVolume.Rushing_Att
RushingVolume = RushingVolume.sort_values(['RushAtt_G', 'Yds_Rush', 'TD_Rush'], ascending = [0,0,0])


ReceivingVolume = (
    PlayerData
    [['TeamName', 'Player', 'Pos', 'G', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD']]
    .groupby(['TeamName', 'Player','Pos'])
    .sum()
    .reset_index()
)
ReceivingVolume['TargetTotal'] = (ReceivingVolume
                                 .merge(
                                     ReceivingVolume
                                     .groupby('TeamName')
                                     .Receiving_Tgt
                                     .sum()
                                     .reset_index()
                                     .rename(columns = {'Receiving_Tgt':'TargetTotal'}), on = 'TeamName')
                                 ['TargetTotal'])
ReceivingVolume['TargetShare'] = (ReceivingVolume.Receiving_Tgt )/ ReceivingVolume.TargetTotal
ReceivingVolume['Target_G'] = ReceivingVolume.Receiving_Tgt/ReceivingVolume.G
ReceivingVolume['RecPct'] = ReceivingVolume.Receiving_Rec/ReceivingVolume.Receiving_Tgt
ReceivingVolume['Yds_Rec'] = ReceivingVolume.Receiving_Yds/ReceivingVolume.Receiving_Rec
ReceivingVolume['TD_Rec'] = ReceivingVolume.Receiving_TD/ReceivingVolume.Receiving_Rec

ReceivingVolume = ReceivingVolume.sort_values(['Target_G', 'RecPct', 'Yds_Rec', 'TD_Rec'], ascending = [0,0,0,0])

In [4]:
FantasyStats2019 = PassingVolume.merge(RushingVolume.merge(ReceivingVolume))
FantasyStats2019 = FantasyStats2019[['TeamName', 'Player', 'Pos', 
 'G', 'PassAttTotal', 'RushTotal', 'TargetTotal',
 'PassShare', 'RushShare', 'TargetShare',
 'PassAtt_G', 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att', 
 'Att', 'Cmp', 'Yds', 'TD', 'Int',
 'RushAtt_G', 'Yds_Rush', 'TD_Rush', 
 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD',
 'Target_G', 'RecPct', 'Yds_Rec', 'TD_Rec', 
 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD']]

Columns = FantasyStats2019.columns 
Category = ['Info', 'Info', 'Info', 
 'Volume', 'Volume', 'Volume', 'Volume',
 'Opportunity', 'Opportunity', 'Opportunity',
 'PassEfficiency', 'PassEfficiency', 'PassEfficiency', 'PassEfficiency', 'PassEfficiency', 
 'PassProduction', 'PassProduction', 'PassProduction', 'PassProduction', 'PassProduction',
 'RushEfficiency', 'RushEfficiency', 'RushEfficiency', 
 'RushProduction', 'RushProduction', 'RushProduction',
 'RecevingEfficiency', 'RecevingEfficiency', 'RecevingEfficiency', 'RecevingEfficiency', 
 'ReceivingProduction', 'ReceivingProduction', 'ReceivingProduction', 'ReceivingProduction']

NewIndex = pd.DataFrame({'Category':Category, 'Columns':Columns})
NewIndex = [Category,Columns]

FantasyStats2019.columns = pd.MultiIndex.from_arrays(NewIndex)
FantasyStats2019

Info                      Volume                         \
                 TeamName           Player Pos      G PassAttTotal RushTotal   
0       Carolina Panthers       Cam Newton  QB      2        633.0     379.0   
1         Atlanta Falcons        Matt Ryan  QB     15        683.0     336.0   
2      Cincinnati Bengals      Andy Dalton  QB     13        616.0     385.0   
3        Los Angeles Rams       Jared Goff  QB     16        630.0     395.0   
4    Tampa Bay Buccaneers   Jameis Winston  QB     16        630.0     394.0   
..                    ...              ...  ..    ...          ...       ...   
499   Washington Redskins  Paul Richardson  WR     10        479.0     294.0   
500   Washington Redskins     Robert Davis  WR      3        479.0     294.0   
501   Washington Redskins   Terry McLaurin  WR     14        479.0     294.0   
502   Washington Redskins       Trey Quinn  WR     12        479.0     294.0   
503   Washington Redskins     Vernon Davis  TE      4        479.0     294.0   

                Opportunity                        ... RushProduction  \
    TargetTotal   PassShare RushShare TargetShare  ...    Rushing_Yds   
0         591.0    0.140600  0.013193    0.000000  ...           -2.0   
1         617.0    0.901903  0.101190    0.000000  ...          147.0   
2         576.0    0.857143  0.083117    0.001736  ...           73.0   
3         605.0    0.993651  0.083544    0.000000  ...           40.0   
4         556.0    0.993651  0.149746    0.000000  ...          250.0   
..          ...         ...       ...         ...  ...            ...   
499       392.0    0.000000  0.003401    0.107143  ...            0.0   
500       392.0    0.000000  0.000000    0.005102  ...            0.0   
501       392.0    0.000000  0.000000    0.237245  ...            0.0   
502       392.0    0.000000  0.000000    0.119898  ...            0.0   
503       392.0    0.000000  0.000000    0.048469  ...            0.0   

               RecevingEfficiency                                 \
    Rushing_TD           Target_G    RecPct    Yds_Rec    TD_Rec   
0          0.0           0.000000       NaN        NaN       NaN   
1          1.0           0.000000       NaN        NaN       NaN   
2          4.0           0.076923  1.000000  -4.000000  0.000000   
3          2.0           0.000000       NaN        NaN       NaN   
4          1.0           0.000000       NaN        NaN       NaN   
..         ...                ...       ...        ...       ...   
499        0.0           4.200000  0.666667   8.750000  0.071429   
500        0.0           0.666667  0.500000  11.000000  0.000000   
501        0.0           6.642857  0.623656  15.844828  0.120690   
502        0.0           3.916667  0.553191   7.615385  0.038462   
503        0.0           4.750000  0.526316  12.300000  0.100000   

    ReceivingProduction                                           
          Receiving_Tgt Receiving_Rec Receiving_Yds Receiving_TD  
0                   0.0           0.0           0.0          0.0  
1                   0.0           0.0           0.0          0.0  
2                   1.0           1.0          -4.0          0.0  
3                   0.0           0.0           0.0          0.0  
4                   0.0           0.0           0.0          0.0  
..                  ...           ...           ...          ...  
499                42.0          28.0         245.0          2.0  
500                 2.0           1.0          11.0          0.0  
501                93.0          58.0         919.0          7.0  
502                47.0          26.0         198.0          1.0  
503                19.0          10.0         123.0          1.0  

[504 rows x 34 columns]

In [5]:
FantasyStats2019.to_excel(joinpath(path,r"C:\Users\sdhec\Google Drive\Fantasy Football\Fantasy Football Stats - 2019.xlsx"))

In [6]:
Teams = FantasyStats2019.Info.TeamName.drop_duplicates()
with pd.ExcelWriter(joinpath(path, r'C:\Users\sdhec\Google Drive\Fantasy Football\Fantasy Football 2019.xlsx')) as writer:
    for team in Teams: 
        FantasyStats2019[FantasyStats2019.Info.TeamName == team].to_excel(writer, sheet_name= team)

We want to have: 
- Team Volume: Number of Games, Team Passing Attempts, Team Rushing Attempts, Team Targets
- PLayer Opportunity: Passing Attempts Share, Rushing Share, Target Share
- Player Efficiency metrics. 

Next up is the very same information, but forecasted for the next season: 
- Team Volume: Expected games they will play
- Player Opportunity: Adjust to reflect expected roster utilization
- Player Efficiency: Mostly keep the same, but can adjust to be pessimistic about outliers. 


In [33]:
FantasyStats2020 = (PassingVolume.merge(RushingVolume.merge(ReceivingVolume)).drop(['TeamName', 'Pos'],axis =1))
FantasyStats2020.head()

,Player,G,Att,Cmp,Yds,TD,Int,PassAttTotal,PassShare,PassAtt_G,...,Receiving_Tgt,Receiving_Rec,Receiving_Yds,Receiving_TD,TargetTotal,TargetShare,Target_G,RecPct,Yds_Rec,TD_Rec
0,Cam Newton,2,89.0,50.0,572.0,0.0,1.0,633.0,0.140600,44.500000,...,0.0,0.0,0.0,0.0,591.0,0.000000,0.000000,NaN,NaN,NaN
1,Matt Ryan,15,616.0,408.0,4466.0,26.0,14.0,683.0,0.901903,41.066667,...,0.0,0.0,0.0,0.0,617.0,0.000000,0.000000,NaN,NaN,NaN
2,Andy Dalton,13,528.0,314.0,3494.0,16.0,14.0,616.0,0.857143,40.615385,...,1.0,1.0,-4.0,0.0,576.0,0.001736,0.076923,1.0,-4.0,0.0
3,Jared Goff,16,626.0,394.0,4638.0,22.0,16.0,630.0,0.993651,39.125000,...,0.0,0.0,0.0,0.0,605.0,0.000000,0.000000,NaN,NaN,NaN
4,Jameis Winston,16,626.0,380.0,5109.0,33.0,30.0,630.0,0.993651,39.125000,...,0.0,0.0,0.0,0.0,556.0,0.000000,0.000000,NaN,NaN,NaN


In [34]:
FantasyStats2020[FantasyStats2020.Player.str.contains('Kenyan Drake')]

,Player,G,Att,Cmp,Yds,TD,Int,PassAttTotal,PassShare,PassAtt_G,...,Receiving_Tgt,Receiving_Rec,Receiving_Yds,Receiving_TD,TargetTotal,TargetShare,Target_G,RecPct,Yds_Rec,TD_Rec
108,Kenyan Drake,8,0.0,0.0,0.0,0.0,0.0,554.0,0.0,0.0,...,35.0,28.0,171.0,0.0,525.0,0.066667,4.375,0.800000,6.107143,0.0
326,Kenyan Drake,6,0.0,0.0,0.0,0.0,0.0,615.0,0.0,0.0,...,33.0,22.0,174.0,0.0,557.0,0.059246,5.500,0.666667,7.909091,0.0


In [39]:
#FantasyStats2020 = (PassingVolume.merge(RushingVolume.merge(ReceivingVolume)).drop(['TeamName', 'Pos'],axis =1))

FantasyStats2020 = (PassingVolume.merge(RushingVolume.merge(ReceivingVolume)))
FantasyStats2020 = FantasyStats2020.merge(Roster[Roster.Pos.isin(['QB', 'RB', 'WR', 'TE'])][['TeamName', 'Player', 'Pos']],how = 'right')
FantasyStats2020 = FantasyStats2020[['TeamName', 'Player', 'Pos', 
 'G', 'PassAttTotal', 'RushTotal', 'TargetTotal',
 'PassShare', 'RushShare', 'TargetShare',
 'CmpPct', 'Yds_Cmp', 'TD_Cmp', 'Int_Att', 
 'PassAtt_G', 'Att', 'Cmp', 'Yds', 'TD', 'Int',
 'Yds_Rush', 'TD_Rush', 
 'RushAtt_G', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD',
 'RecPct', 'Yds_Rec', 'TD_Rec', 
 'Target_G', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD']].sort_values(['TeamName', 'Pos', 'Player'])
Columns = FantasyStats2020.columns 
Category = ['Info', 'Info', 'Info', 
 'Volume', 'Volume', 'Volume', 'Volume',
 'Opportunity', 'Opportunity', 'Opportunity',
 'PassEfficiency', 'PassEfficiency', 'PassEfficiency', 'PassEfficiency', 
 'PassProduction', 'PassProduction', 'PassProduction', 'PassProduction', 'PassProduction', 'PassProduction',
 'RushEfficiency', 'RushEfficiency', 
 'RushProduction', 'RushProduction', 'RushProduction', 'RushProduction',
 'RecevingEfficiency', 'RecevingEfficiency', 'RecevingEfficiency', 
 'ReceivingProduction', 'ReceivingProduction', 'ReceivingProduction', 'ReceivingProduction', 'ReceivingProduction']

NewIndex = pd.DataFrame({'Category':Category, 'Columns':Columns})
NewIndex = [Category,Columns]

FantasyStats2020.columns = pd.MultiIndex.from_arrays(NewIndex)
FantasyStats2020 = FantasyStats2020[~FantasyStats2020.Info.TeamName.isnull()]
FantasyStats2020

Info                      Volume                         \
                TeamName           Player Pos      G PassAttTotal RushTotal   
33     Arizona Cardinals    Brett Hundley  QB    3.0        554.0     394.0   
773    Arizona Cardinals  Chris Streveler  QB    NaN          NaN       NaN   
772    Arizona Cardinals    Drew Anderson  QB    NaN          NaN       NaN   
10     Arizona Cardinals     Kyler Murray  QB   16.0        554.0     394.0   
55     Arizona Cardinals    Chase Edmonds  RB   13.0        554.0     394.0   
..                   ...              ...  ..    ...          ...       ...   
585  Washington Redskins     Jordan Veasy  WR    NaN          NaN       NaN   
50   Washington Redskins    Kelvin Harmon  WR   16.0        479.0     294.0   
53   Washington Redskins      Steven Sims  WR   16.0        479.0     294.0   
283  Washington Redskins   Terry McLaurin  WR   14.0        479.0     294.0   
284  Washington Redskins       Trey Quinn  WR   12.0        479.0     294.0   

                Opportunity                        ... RushProduction  \
    TargetTotal   PassShare RushShare TargetShare  ...    Rushing_Yds   
33        525.0    0.019856  0.017766    0.000000  ...           41.0   
773         NaN         NaN       NaN         NaN  ...            NaN   
772         NaN         NaN       NaN         NaN  ...            NaN   
10        525.0    0.978339  0.236041    0.000000  ...          544.0   
55        525.0    0.000000  0.152284    0.040000  ...          303.0   
..          ...         ...       ...         ...  ...            ...   
585         NaN         NaN       NaN         NaN  ...            NaN   
50        392.0    0.002088  0.000000    0.112245  ...            0.0   
53        392.0    0.002088  0.030612    0.142857  ...           85.0   
283       392.0    0.000000  0.000000    0.237245  ...            0.0   
284       392.0    0.000000  0.000000    0.119898  ...            0.0   

               RecevingEfficiency                      ReceivingProduction  \
    Rushing_TD             RecPct    Yds_Rec    TD_Rec            Target_G   
33         0.0                NaN        NaN       NaN            0.000000   
773        NaN                NaN        NaN       NaN                 NaN   
772        NaN                NaN        NaN       NaN                 NaN   
10         4.0                NaN        NaN       NaN            0.000000   
55         4.0           0.571429   8.750000  0.083333            1.615385   
..         ...                ...        ...       ...                 ...   
585        NaN                NaN        NaN       NaN                 NaN   
50         0.0           0.681818  12.166667  0.000000            2.750000   
53         1.0           0.607143   9.117647  0.117647            3.500000   
283        0.0           0.623656  15.844828  0.120690            6.642857   
284        0.0           0.553191   7.615385  0.038462            3.916667   

                                                            
    Receiving_Tgt Receiving_Rec Receiving_Yds Receiving_TD  
33            0.0           0.0           0.0          0.0  
773           NaN           NaN           NaN          NaN  
772           NaN           NaN           NaN          NaN  
10            0.0           0.0           0.0          0.0  
55           21.0          12.0         105.0          1.0  
..            ...           ...           ...          ...  
585           NaN           NaN           NaN          NaN  
50           44.0          30.0         365.0          0.0  
53           56.0          34.0         310.0          4.0  
283          93.0          58.0         919.0          7.0  
284          47.0          26.0         198.0          1.0  

[775 rows x 34 columns]

In [40]:
Teams = FantasyStats2020.Info.TeamName.drop_duplicates()
with pd.ExcelWriter(joinpath(path, r'C:\Users\sdhec\Google Drive\Fantasy Football\Fantasy Football 2020.xlsx')) as writer:
    for team in Teams: 
        FantasyStats2020[FantasyStats2020.Info.TeamName == team].to_excel(writer, sheet_name= team)